In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import re

class Automata(object):
    # Expresiones regulares para identificar los elementos de la tupla
    sigma_pattern = r'(sigma=)\{([^\s,]+[^\s]*)\}'
    Q_pattern = r'(Q=)\{([q0-9,]+)\}'
    f_pattern = r'(f=)\{([^\}]+)\}'
    q0_pattern = r'(q0=)(q[0-9]+)'
    F_pattern = r'(F=)\{([q0-9,]+)\}'
    test_pattern = r'(test=)\[([^\]]+)\]'
    expected_pattern = r'(expected=)\[((accepted|rejected)(,(accepted|rejected))*)\]'

    """
    Clase que implementa un Autómata Finito Determinista (AFD).
    Un AFD es una máquina de estados que procesa cadenas de símbolos y determina si pertenecen a un lenguaje.
    """
    def __init__(self, sigma=None, states=None, functions=None, initial_state=None, final_states=None):
        """
        Constructor de la clase AFD.
        Parámetros:
        - sigma: alfabeto del autómata
        - states: conjunto de estados
        - functions: funciones de transición
        - initial_state: estado inicial
        - final_states: estados finales
        """
        self._sigma = None          # Alfabeto del autómata
        self._states = None         # Conjunto de estados
        self._functions = None      # Funciones de transición
        self._initial_state = None  # Estado inicial
        self._final_states = None   # Estados finales
        self._file_lines = None     # Variable para almacenar las líneas del archivo

        # Inicialización de atributos si se proporcionan valores
        if sigma:
            self.sigma = sigma
        if states:
            self.states = states
        if functions:
            self.functions = functions
        if initial_state:
            self.initial_state = initial_state
        if final_states:
            self.final_states = final_states

    @property
    def sigma(self):
        """Getter para el alfabeto del autómata"""
        return self._sigma

    @sigma.setter
    def sigma(self, file_lines):
        """
        Setter para el alfabeto del autómata.
        Valida que sigma sea una lista no vacía de símbolos.
        Valida que sigma contenga solo símbolos individuales.
        """
        match = re.search(self.sigma_pattern, file_lines)
        if not match:
            raise ValueError("No se encontró el alfabeto (sigma)")
        sigma = set(match.group(2).split(','))
        if sigma is None:
            raise ValueError("El alfabeto (sigma) no puede estar vacío")
        if not all(len(symbol) == 1 for symbol in sigma):
            raise ValueError("Sigma debe contener solo símbolos individuales.")
        self._sigma = sigma

    @property
    def states(self):
        """Getter para los estados del autómata"""
        return self._states

    @states.setter
    def states(self, file_lines):
        """
        Setter para los estados del autómata.
        Genera estados con nombres q1, q2, ..., qn donde n es el número proporcionado.
        """
        match = re.search(self.Q_pattern, file_lines)
        if not match:
            raise ValueError("No se encontraron los estados (Q)")
        Q = set(match.group(2).split(','))
        if Q is None:
            raise ValueError("Los estados no pueden estar vacíos")
        if not all(re.match(r'q[0-9]+', state) for state in Q):
            raise ValueError("Q contiene estados no válidos.")
        self._states = Q

    @property
    def functions(self):
        """Getter para las funciones de transición"""
        return self._functions

    @functions.setter
    def functions(self, file_lines):
        """
        Setter para las funciones de transición.
        Procesa las transiciones en formato 'q1,símbolo,q2' y las almacena en un diccionario.
        """
        match = re.search(self.f_pattern, file_lines)
        if not match:
            raise ValueError("No se encontraron las funciones de transición (f)")
        f = match.group(2).split(',')
        if f is None:
            raise ValueError("Las funciones de transición no pueden estar vacías")

        transitions = {}
        # Validar transiciones
        for transition in f:
            # Suponiendo que cada transición está en el formato "(q0 0)->q1"
            match = re.match(r'\(([q0-9]+)\s+([^\)])\)->([q0-9]+)', transition.strip())
            if match:
                state = match.group(1)
                symbol = match.group(2)
                next_state = match.group(3)

                if state not in self._states:
                    raise ValueError(f"El estado {state} en la transición no está en Q.")
                if symbol not in self._sigma:
                    raise ValueError(f"El símbolo {symbol} en la transición no está en sigma.")
                if next_state not in self._states:
                    raise ValueError(f"El estado siguiente {next_state} en la transición no está en Q.")
                if state not in transitions:
                    transitions[state] = {}

                transitions[state][symbol] = next_state
            else:
                raise ValueError(f"Transición no válida: {transition}")
        self._functions = transitions

    @property
    def initial_state(self):
        """Getter para el estado inicial"""
        return self._initial_state

    @initial_state.setter
    def initial_state(self, file_lines):
        """
        Setter para el estado inicial.
        Valida que el estado inicial sea un estado válido del autómata.
        """
        match = re.search(self.q0_pattern, file_lines)
        if not match:
            raise ValueError("No se encontró el estado inicial (q0)")
        q0 = match.group(2)
        if q0 is None:
            raise ValueError("El estado inicial no puede estar vacío")
        if q0 not in self._states:
            raise ValueError("El estado inicial q0 no está en Q.")
        self._initial_state = q0

    @property
    def final_states(self):
        """Getter para los estados finales"""
        return self._final_states

    @final_states.setter
    def final_states(self, file_lines):
        """
        Setter para los estados finales.
        Valida que todos los estados finales sean estados válidos del autómata.
        """
        match = re.search(self.F_pattern, file_lines)
        if not match:
            raise ValueError("No se encontraron los estados finales (F)")
        F = set(match.group(2).split(','))
        if F is None:
            raise ValueError("Los estados finales no pueden estar vacíos")
        if not all(state in self._states for state in F):
            raise ValueError("F contiene estados finales no válidos.")
        self._final_states = F

    def evaluate_string(self, file_lines):
        """
        Procesa una cadena de entrada para determinar si es aceptada por el autómata.
        Retorna una lista de "accepted" o "rejected" para cada subcadena procesada.

        Retorna:
        - Lista con las cadenas de prueba
        - Lista con los resultados esperados
        - Lista con el resultado ("accepted"/"rejected") para cada subcadena
        """

        match_test = re.search(self.test_pattern, file_lines)
        match_expected = re.search(self.expected_pattern, file_lines)
        if not match_test or not match_expected:
            raise ValueError("No se encontraron las listas de prueba o resultados esperados")

        test = [string.strip() for string in match_test.group(2).split(',')]
        expected = [value.strip() for value in match_expected.group(2).split(',')]

        if len(expected) != len(test):
            raise ValueError("La lista de resultados esperados y la lista de cadenas de prueba no tienen la misma longitud")
        if not expected:
            raise ValueError("Los resultados esperados no pueden estar vacíos")
        if not test:
            raise ValueError("Las cadenas de prueba no pueden estar vacías")

        output = []
        for string in test:
            current_state = self._initial_state
            for symbol in string:
                if symbol in self._sigma:
                    if current_state in self._functions.keys() and symbol in self._functions[current_state].keys():
                        current_state = self._functions[current_state][symbol]
                    else:
                        break
                else:
                    break
            if current_state in self._final_states:
                output.append("accepted")
            else:
                output.append("rejected")
        return test, expected, output

    def load_from_file(self, file_path):
        """
        Carga las líneas de un archivo y las almacena en una variable para su uso posterior.

        Parámetros:
        - file_path: ruta al archivo que contiene la definición del autómata
        """
        try:
            with open(file_path, 'r') as file:
                self._file_lines = file.read()
        except FileNotFoundError:
            print(f"El archivo {file_path} no fue encontrado.")
        except Exception as e:
            print(f"Ocurrió un error al leer el archivo: {e}")
        return self._file_lines

    def show_automata(self):
        """
        Muestra el autómata en la consola.
        """
        print(f"sigma={self.sigma}")
        print(f"Q={self.states}")
        print(f"f={self.functions}")
        print(f"q0={self.initial_state}")
        print(f"F={self.final_states}")
        print(f"test={self.evaluate_string(self._file_lines)[0]}")
        print(f"expected={self.evaluate_string(self._file_lines)[1]}")
        print(f"obtained={self.evaluate_string(self._file_lines)[2]}")


In [ ]:
def main():
  try:
      automata = Automata()
      file_lines = automata.load_from_file("/content/drive/MyDrive/automata_correct.def")
      automata.sigma = file_lines
      automata.states = file_lines
      automata.functions = file_lines
      automata.initial_state = file_lines
      automata.final_states = file_lines
      automata.show_automata()
  except Exception as e:
      print(e)

# Punto de entrada del programa
if __name__ == "__main__":
    main()

sigma={'1', '0'}
Q={'q1', 'q2'}
f={'q1': {'0': 'q1', '1': 'q2'}, 'q2': {'0': 'q1', '1': 'q2'}}
q0=q1
F={'q2'}
test=['1', '0', '11', '101', '01010', '000']
expected=['accepted', 'rejected', 'accepted', 'accepted', 'rejected', 'rejected']
obtained=['accepted', 'rejected', 'accepted', 'accepted', 'rejected', 'rejected']
